<a href="https://colab.research.google.com/github/Triniti0/klasifikasi-penyakit-daun-cabai/blob/main/Comparison_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
import json
import time
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
BASE_DIR = "/content/drive/MyDrive/Seminar Hasil/Experiments"

experiment_folders = [
    "Feature_Extraction",
    "Partial_FineTuning",
    "Progressive_FineTuning",
    "Two_Stage_Training"
]

In [ ]:
IMG_SIZE = 224

def load_test_dataset():

    test_ds = tf.keras.utils.image_dataset_from_directory(
        "/content/drive/MyDrive/Seminar Hasil/dataset_processed/test",
        image_size=(IMG_SIZE, IMG_SIZE),
        batch_size=32,
        shuffle=False
    )

    def preprocess(x, y):
        return preprocess_input(x), y

    test_ds = test_ds.map(preprocess)

    return test_ds


In [ ]:
def measure_inference_time(model, test_ds, repetitions=30):

    # Ambil 1 gambar saja
    for images, labels in test_ds.take(1):
        single_image = images[0:1]

    # Warm-up GPU
    for _ in range(5):
        _ = model.predict(single_image, verbose=0)

    times = []

    for _ in range(repetitions):
        start = time.time()
        _ = model.predict(single_image, verbose=0)
        end = time.time()
        times.append(end - start)

    return np.mean(times)

In [ ]:
comparison_results = []
test_ds = load_test_dataset()

for folder in experiment_folders:

    info_path = os.path.join(BASE_DIR, folder, "best_model_info.json")

    if not os.path.exists(info_path):
        print("Model tidak ditemukan untuk:", folder)
        continue

    with open(info_path, "r") as f:
        info = json.load(f)

    print("\nEvaluating:", folder)

    model_path = info["Model Path"]
    # Correct capitalization inconsistency if present
    if "Seminar hasil" in model_path and "Seminar Hasil" in BASE_DIR:
        model_path = model_path.replace("Seminar hasil", "Seminar Hasil")

    model = tf.keras.models.load_model(model_path)

    # === TEST ACCURACY ===
    test_loss, test_acc = model.evaluate(test_ds, verbose=0)

    # === F1 SCORE ===
    y_true = []
    y_pred = []

    for images, labels in test_ds:
        preds = model.predict(images, verbose=0)
        y_true.extend(labels.numpy())
        y_pred.extend(np.argmax(preds, axis=1))

    macro_f1 = f1_score(y_true, y_pred, average='macro')
    weighted_f1 = f1_score(y_true, y_pred, average='weighted')

    # === INFERENCE TIME ===
    inference_time = measure_inference_time(model, test_ds)

    # === MODEL SIZE ===
    size_mb = os.path.getsize(model_path) / (1024*1024)

    comparison_results.append({
        "Experiment": folder,
        "Test Accuracy": test_acc,
        "Macro F1": macro_f1,
        "Weighted F1": weighted_f1,
        "Inference Time (s)": inference_time,
        "Model Size (MB)": size_mb
    })


Found 620 files belonging to 8 classes.

Evaluating: Feature_Extraction



Evaluating: Partial_FineTuning



Evaluating: Progressive_FineTuning



Evaluating: Two_Stage_Training


In [ ]:
comparison_df = pd.DataFrame(comparison_results)

comparison_df = comparison_df.sort_values(
    by=["Test Accuracy"],
    ascending=False
)

comparison_df.reset_index(drop=True)

,Experiment,Test Accuracy,Macro F1,Weighted F1,Inference Time (s),Model Size (MB)
0,Partial_FineTuning,0.996774,0.990192,0.996777,0.074178,17.275917
1,Progressive_FineTuning,0.995161,0.986640,0.995154,0.074025,26.831291
2,Feature_Extraction,0.990323,0.981327,0.990319,0.070745,11.017670
3,Two_Stage_Training,0.990323,0.977437,0.990548,0.112262,17.275879


In [ ]:
# Save the comparison results to a CSV file
output_csv_path = os.path.join(BASE_DIR, "model_comparison_results.csv")
comparison_df.to_csv(output_csv_path, index=False)
print(f"Comparison results saved to {output_csv_path}")

Comparison results saved to /content/drive/MyDrive/Seminar Hasil/Experiments/model_comparison_results.csv
